# Deep Neural Network Comparison

### Load CIFAR Dataset

To obtain the CIFAR10 dataset, go here: https://www.cs.toronto.edu/~kriz/cifar.html

The training data is stored in 5 separate files, and we will alternate between them during training.

In [12]:
#Load necessary libraries
import tensorflow as tf
import numpy as np
import tensorflow.contrib.slim as slim
import input_data
import matplotlib.pyplot as plt
%matplotlib inline
def unpickle(file):
    import pickle
    fo = open(file, 'rb')
    dict = pickle.load(fo)
    fo.close()
    return dict
currentCifar = 1
cifartr = unpickle('train0321.txt')


for i in range(len(cifartr)):
    #print (np.average(bbb[i]))
    #print (np.max(bbb[i]))
    cifartr[i]=cifartr[i]-np.average(cifartr[i])
    cifartr[i]=cifartr[i]/np.std(cifartr[i])
vali=cifartr[73000:73200]
cifartr=cifartr[0:73000]

cifarTla = unpickle('newlabley2.txt')
cifarTla=np.argmax(cifarTla,axis=1)

valila=cifarTla[73000:73200]
cifarTla=cifarTla[0:73000]
print("down")
print(cifarTla)
total_layers = 25 #Specify how deep we want our network
units_between_stride = int(total_layers / 5)+1

down
[1 9 2 ..., 5 3 5]


## ResNet
An implementation of a Residual Network as described in [Identity Mappings in Deep Residual Networks](https://arxiv.org/abs/1603.05027).

In [1]:
def resUnit(input_layer,i):
    with tf.variable_scope("res_unit"+str(i)):
        part1 = slim.batch_norm(input_layer,activation_fn=None)
        part2 = tf.nn.relu(part1)
        part3 = slim.conv2d(part2,64,[3,3],activation_fn=None)
        part4 = slim.batch_norm(part3,activation_fn=None)
        part5 = tf.nn.relu(part4)
        part6 = slim.conv2d(part5,64,[3,3],activation_fn=None)
        output = input_layer + part6
        print(part6.shape)
        return output
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):

  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)
tf.reset_default_graph()

input_layer = tf.placeholder(shape=[None,32,32,1],dtype=tf.float32,name='input')
label_layer = tf.placeholder(shape=[None],dtype=tf.int32)
label_oh = slim.layers.one_hot_encoding(label_layer,10)

layer1 = slim.conv2d(input_layer,64,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(0))
for i in range(5):
    for j in range(units_between_stride):
        print(units_between_stride)
        layer1 = resUnit(layer1,j + (i*units_between_stride))
    layer1 = slim.conv2d(layer1,64,[3,3],stride=[2,2],normalizer_fn=slim.batch_norm,scope='conv_s_'+str(i))
    
top = slim.conv2d(layer1,10,[3,3],normalizer_fn=slim.batch_norm,activation_fn=None,scope='conv_top')
print("top",top.shape)
output = slim.layers.softmax(slim.layers.flatten(top))

loss = tf.reduce_mean(-tf.reduce_sum(label_oh * tf.log(output) + 1e-10, axis=[1]))


NameError: name 'tf' is not defined

### Visualize the network graph
We can call the Tensorflow Board to provide a graphical representation of our network.

In [3]:
# from IPython.display import clear_output, Image, display, HTML

# def strip_consts(graph_def, max_const_size=32):
#     """Strip large constant values from graph_def."""
#     strip_def = tf.GraphDef()
#     for n0 in graph_def.node:
#         n = strip_def.node.add() 
#         n.MergeFrom(n0)
#         if n.op == 'Const':
#             tensor = n.attr['value'].tensor
#             size = len(tensor.tensor_content)
#             if size > max_const_size:
#                 tensor.tensor_content = "<stripped %d bytes>"%size
#     return strip_def

# def show_graph(graph_def, max_const_size=32):
#     """Visualize TensorFlow graph."""
#     if hasattr(graph_def, 'as_graph_def'):
#         graph_def = graph_def.as_graph_def()
#     strip_def = strip_consts(graph_def, max_const_size=max_const_size)
#     code = """
#         <script>
#           function load() {{
#             document.getElementById("{id}").pbtxt = {data};
#           }}
#         </script>
#         <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
#         <div style="height:600px">
#           <tf-graph-basic id="{id}"></tf-graph-basic>
#         </div>
#     """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

#     iframe = """
#         <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
#     """.format(code.replace('"', '&quot;'))
#     display(HTML(iframe))

In [4]:
# show_graph(tf.get_default_graph().as_graph_def())


## Training

In [ ]:
batch_size = 100
global_step = tf.Variable(0, trainable=False)
#boundaries = [int(73000*2/batchsize*3+100000/batchsize*0),int(73000*2/batchsize*3+100000/batchsize*1),int(73000*2/batchsize*3+100000/batchsize*3),int(73000*2/batchsize*3+100000/batchsize*6)]
boundaries = [int(73000/batch_size*3),int(73000/batch_size*6),int(73000/batch_size*6),int(73000/batch_size*10)]
#values = [0.2e-4,0.18e-4,0.17e-4,0.16e-4,0.15e-4] #i changed it form 0.4e-4 to 1.5e-4
values = [1.5e-4,1e-4,0.8e-4,0.6e-4,0.4e-4]
lr = tf.train.piecewise_constant(global_step, boundaries, values)


trainer = tf.train.AdamOptimizer(learning_rate=lr)
update = trainer.minimize(loss,global_step=global_step)

init = tf.global_variables_initializer()

currentCifar = 1
total_steps = 73000/100*10
l = []
a = []
aT = []
with tf.Session() as sess:
    sess.run(init)
    saver = tf.train.Saver()
    #saver.restore(sess, "backup/resnigh1.ckpt");print("load")
    i = 0
    draw = range(10000)
    while i < total_steps:
        if i % (10000/batch_size) != 0:
            batch_index = np.random.choice(draw,size=batch_size,replace=False)
        else:
            draw = range(10000)
#             if currentCifar == 5:
#                 currentCifar = 1
#                 print ("Switched CIFAR set to " + str(currentCifar))
#             else:
#                 currentCifar = currentCifar + 1
#                 print ("Switched CIFAR set to " + str(currentCifar))
#             cifar = unpickle('./cifar10/data_batch_'+str(currentCifar))
            batch_index = np.random.choice(draw,size=batch_size,replace=False)
        x = cifartr[batch_index]
        x = np.reshape(x,[batch_size,32,32,1],order='F')
        y = np.reshape(np.array(cifarTla)[batch_index],[batch_size,1])
        _,lossA,yP,LO = sess.run([update,loss,output,label_oh],feed_dict={input_layer:x,label_layer:np.hstack(y)})
        accuracy = np.sum(np.equal(np.hstack(y),np.argmax(yP,1)))/float(len(y))
        l.append(lossA)
        a.append(accuracy)
        if i % 10 == 0: 
            print ("Step: " + str(i) + " Loss: " + str(lossA) + " Accuracy: " + str(accuracy))
            xT=np.reshape(vali,[-1,32,32,1],order='F')
            yT = np.reshape(np.array(valila),[200])
            lossT,yP = sess.run([loss,output],feed_dict={input_layer:xT,label_layer:yT})
            valiaccuracy = np.sum(np.equal(yT,np.argmax(yP,1)))/float(len(yT))
            
            print("vali is ",valiaccuracy)
            print("learning rate is",lr)
#         if i % 100 == 0: 
#             point = np.random.randint(0,10000-500)
#             xT = cifarT['data'][point:point+500]
#             xT = np.reshape(xT,[500,32,32,3],order='F')
#             xT = (xT/256.0)
#             xT = (xT - np.mean(xT,axis=0)) / np.std(xT,axis=0)
#             yT = np.reshape(np.array(cifarT['labels'])[point:point+500],[500])
#             lossT,yP = sess.run([loss,output],feed_dict={input_layer:xT,label_layer:yT})
#             accuracy = np.sum(np.equal(yT,np.argmax(yP,1)))/float(len(yT))
#             aT.append(accuracy)
#             print( "Test set accuracy: " + str(accuracy))
        i+= 1
    saver.save(sess, "backup/resnigh1.ckpt")
    
#20 second to appear something FOR 5 LAYERES
#5s second to appear the first number for 10 layers

## Results

In [ ]:
lossT,yP = sess.run([loss,output],feed_dict={input_layer:xT,label_layer:yT})

In [ ]:
plt.plot(l) #Plot training loss

In [ ]:
plt.plot(a) #Plot training accuracy

In [ ]:
plt.plot(aT) #Plot test accuracy

In [ ]:
np.max(aT) #Best test accuracy

In [ ]:
trainer = tf.train.AdamOptimizer(learning_rate=1.5e-4)
# update = trainer.minimize(loss)

# init = tf.global_variables_initializer()
# batch_size = 100
# currentCifar = 1
# total_steps = 73000/100
# l = []
# a = []
# aT = []
# sess = tf.InteractiveSession()
# sess.run(init)
# saver = tf.train.Saver()
#     #saver.restore(sess, "codebackup/model4.ckpt")
# i = 0
# draw = range(10000)
# while i < total_steps:
#     if i % (10000/batch_size) != 0:
#         batch_index = np.random.choice(draw,size=batch_size,replace=False)
#     else:
#         draw = range(10000)
# #             if currentCifar == 5:
# #                 currentCifar = 1
# #                 print ("Switched CIFAR set to " + str(currentCifar))
# #             else:
# #                 currentCifar = currentCifar + 1
# #                 print ("Switched CIFAR set to " + str(currentCifar))
# #             cifar = unpickle('./cifar10/data_batch_'+str(currentCifar))
#         batch_index = np.random.choice(draw,size=batch_size,replace=False)
#     x = cifartr[batch_index]
#     x = np.reshape(x,[batch_size,32,32,1],order='F')
# #         x = (x)
# #         x = (x - np.mean(x,axis=0)) / np.std(x,axis=0)
#     y = np.reshape(np.array(cifarTla)[batch_index],[batch_size,1])
#     _,lossA,yP,LO = sess.run([update,loss,output,label_oh],feed_dict={input_layer:x,label_layer:np.hstack(y)})
#     accuracy = np.sum(np.equal(np.hstack(y),np.argmax(yP,1)))/float(len(y))
#     l.append(lossA)
#     a.append(accuracy)
#     if i % 10 == 0: print ("Step: " + str(i) + " Loss: " + str(lossA) + " Accuracy: " + str(accuracy))
# #         if i % 100 == 0: 
# #             point = np.random.randint(0,10000-500)
# #             xT = cifarT['data'][point:point+500]
# #             xT = np.reshape(xT,[500,32,32,3],order='F')
# #             xT = (xT/256.0)
# #             xT = (xT - np.mean(xT,axis=0)) / np.std(xT,axis=0)
# #             yT = np.reshape(np.array(cifarT['labels'])[point:point+500],[500])
# #             lossT,yP = sess.run([loss,output],feed_dict={input_layer:xT,label_layer:yT})
# #             accuracy = np.sum(np.equal(yT,np.argmax(yP,1)))/float(len(yT))
# #             aT.append(accuracy)
# #             print( "Test set accuracy: " + str(accuracy))
#     i+= 1
#10 second to appear something